In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [21]:
import import_ipynb
import os
os.chdir('/Users/kongdowoung/Desktop/WatermelonTheBest/')
from boundary import boundary 
os.chdir('/Users/kongdowoung/Desktop/WatermelonTheBest/initial_vrp')

In [22]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')
bus_name_lst = ['강남역','학동역','현대아파트','신당동.청구역','동묘앞','안암오거리','서초동삼성아파트','방배역','사당역13번출구','남성초등학교','언북중학교입구','세관앞','신구중학교','금옥초등학교.금호대우아파트','장충체육관앞','대광고등학교앞','서초역2번출구','청호나이스','방배동신동아아파트','서울고사거리','방배동래미안아파트','서울교통공사','구름다리','이수역','사당역']

f = open("grouped_bus_station.pickle", "rb")
grouping  = pickle.load(f)
f.close()
route = grouping['bus_station'][0] #기존 경로 노드

my_boundary = boundary()
possible_bs_lst = my_boundary.all_possible_node(time_matrix, bus_name_lst, route)
possible_bs_lst = [i for i in possible_bs_lst.keys()] #기존 경로 기준 승하차 가능한 노드

import random 
pair = random.sample(possible_bs_lst,2) #최종 결정된 새로운 수요 노드


final_route = route+pair #최종 노드
name_lst2 = [bus_name_lst[i] for i in final_route]

In [23]:
route

[0, 7, 8, 9, 6]

In [24]:
pair

[18, 19]

In [25]:
[bus_name_lst[i] for i in pair]

['방배동신동아아파트', '서울고사거리']

2. pickup_delivery location pair 구하기 (For each pair, the first entry is index of the pickup location, and the second is the index of the delivery location.)

In [26]:
time_matrix = pd.DataFrame(time_matrix)
time_matrix_0 = time_matrix[final_route].loc[final_route] #index 바꾸기
time_matrix_0 = time_matrix_0.to_numpy().tolist()
time_matrix_0

[[0, 601, 936, 1038, 556, 535, 517],
 [727, 0, 276, 564, 318, 147, 140],
 [979, 359, 0, 425, 533, 506, 499],
 [929, 382, 303, 0, 483, 511, 482],
 [506, 316, 549, 809, 0, 250, 233],
 [683, 66, 299, 586, 360, 0, 96],
 [656, 94, 327, 614, 264, 28, 0]]

In [27]:
np.argmax(time_matrix_0[0])

3

In [28]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import datetime


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = time_matrix_0
    data['demands'] = [0,1,1,1,1,1,1] #demand 바꾸기
    data['vehicle_capacities'] = [5]
    data['num_vehicles'] = 1

    #[0, 7, 8, 9, 6] [0,1,2,3,4]
    #[18, 19] [5,6]

    data['pickups_deliveries'] = [[1,2],[5,6]]

    data['starts'] = [0]
    data['ends'] = [int(np.argmax(time_matrix_0[0]))]

    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)

            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst2[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        node_index = manager.IndexToNode(index)
        plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst2[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
        plan_output += '서비스 종료\n\n'
        plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('\n')
    print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
    print('Total load of all routes: {}'.format(total_load))
   

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['starts'], data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        8000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # 3-1. Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(int(request[0]))
        delivery_index = manager.NodeToIndex(int(request[1]))
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)

    
  
if __name__ == '__main__':
    main()

Objective: 171397
Route for vehicle 0:

 정류장 이름 : 강남역  (탑승객 수: 5, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 서초동삼성아파트  (탑승객 수: 4, 하차수: 1, 소요시간: 0:09:16) 
    ==>     정류장 이름 : 방배동신동아아파트  (탑승객 수: 3, 하차수: 1, 소요시간: 0:04:10) 
    ==>     정류장 이름 : 서울고사거리  (탑승객 수: 2, 하차수: 1, 소요시간: 0:01:36) 
    ==>     정류장 이름 : 방배역  (탑승객 수: 1, 하차수: 1, 소요시간: 0:01:34) 
    ==>     정류장 이름 : 사당역13번출구  (탑승객 수: 0, 하차수: 1, 소요시간: 0:04:36) 
    ==>     정류장 이름 : 남성초등학교  (탑승객 수: 0, 하차수: 1, 소요시간: 0:07:05) 
    ==>    서비스 종료

Duration of the route: 0:21:12분
Load of the route: 5



Total Duration of all routes: 0:28:17m
Total load of all routes: 5
